In [91]:
import sys
import gc
import time
import numpy as np
import psutil
import pandas as pd
import sqlite3

%pylab inline
%matplotlib inline
#import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import concurrent.futures
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LogNorm
from decimal import *
from importlib import reload
from scipy.ndimage import zoom 
from astropy.table import Table
from astropy.io import fits

#needed for reload
import procedures
reload (procedures)
#import without needing procedures.module
from procedures import *

print('done')

Populating the interactive namespace from numpy and matplotlib
done


/export/data1/Workspace/GitHub_Python_plots/py3env/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['subplot', 'grid', 'ylim', 'xlim']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [109]:
#DirName_MR = '/Users/BrunoHenriques/Desktop/Work/SAM/Hen15/GalTree/'
#Datadir = './data/'
DirName_MR = '/net/bootes/scratch-ssd/SAM/test1/MR/'
Datadir = '/net/bootes/export/data1/data/'

#to force python to free allocated memory
G_MR = None
gc.collect()
    
FirstFile = 40
LastFile =  40

Hubble_h      = 0.673
BoxSize_MR    = 500.* 0.960558 #full MR 
MaxTreeFiles  = 512
Volume_MR = (BoxSize_MR**3.0) * (LastFile - FirstFile + 1) / MaxTreeFiles / Hubble_h**3

print('Reading started')

#from LGalaxies_tree_Henriques2015a_struct import LGalaxiesStruct
#from LGalaxies_tree_Henriques2015a_struct import PropertiesToRead      
from LGalaxies_tree_Henriques2016a_Elements_Rings_struct_uni_dimensional import LGalaxiesStruct
from LGalaxies_tree_Henriques2016a_Elements_Rings_struct_uni_dimensional import PropertiesToRead
 
(G_MR) = read_tree(DirName_MR,FirstFile,LastFile,PropertiesToRead,LGalaxiesStruct)               
    
#release unused memory
gc.collect()

print("Reading Done, Allocated Memory: %0.2f Gb" % (sys.getsizeof(G_MR)/1e9))

Reading started


Reading Headers

TotNgals= 1827631


File 40  nGals = 1827631
Reading Done, Allocated Memory: 0.29 Gb


### Check structure

In [110]:
#print(G_MR[0])
#print(G_MR[1])
G_MR_sel=G_MR
#print(G_MR.dtype.names)
#print(G_MR.dtype)

df = pd.DataFrame(data=G_MR_sel)
display(df[:3])

,GalID,HaloID,FirstProgGal,NextProgGal,LastProgGal,FOFCentralGal,DescendantGal,MainLeafId,Redshift,Type,...,BlackHoleMass,MassFromInSitu,MassFromMergers,MassFromBursts,Sfr,BulgeSize,StellarDiskRadius,GasDiskRadius,StellarHalfMassRadius,StellarHalfLightRadius
0,40000000000000,40000000000000,40000000000001,-1,40000000004615,40000000000000,-1,40000000000051,-0.103822,0,...,0.141165,27.048784,38.632530,2.052683,0.366554,0.015808,0.009067,0.017384,0.005525,0.005497
1,40000000000001,40000000000001,40000000000002,40000000004547,40000000004546,40000000000001,40000000000000,40000000000051,-0.085948,0,...,0.138554,27.044891,38.569565,2.052385,0.563085,0.015796,0.009068,0.018573,0.005519,0.005489
2,40000000000002,40000000000002,40000000000003,40000000004377,40000000004376,40000000000002,40000000000001,40000000000051,-0.066833,0,...,0.135818,27.039534,38.507469,2.051782,0.530791,0.015790,0.009069,0.013480,0.005517,0.005485


### Write as database file

In [111]:
database_file = DirName_MR + 'GalTree_file40.db'
conn = sqlite3.connect(database_file)
df.to_sql('Henriques2015', conn, if_exists='replace', index=False)
conn.close()

print("Upload complete")

Upload complete


In [112]:
def run_query(database_file, q):
    with sqlite3.connect(database_file) as conn:
        return pd.read_sql(q, conn)
    
def run_command(database_file, q):
    with sqlite3.connect(database_file) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(q)
        
database_file = DirName_MR + 'GalTree_file40.db'

### Check database file and create indexes

In [113]:
query = 'CREATE INDEX GalID_index ON Henriques2015(GalID);'
run_command(database_file, query)

#query = 'Select * from sqlite_master where type="table"'
query = 'PRAGMA table_info(Henriques2015)'
#query = 'select * from Henriques2015 LIMIT 10'

df1 = run_query(database_file, query)

display(df1[:4])

,cid,name,type,notnull,dflt_value,pk
0,0,GalID,INTEGER,0,None,0
1,1,HaloID,INTEGER,0,None,0
2,2,FirstProgGal,INTEGER,0,None,0
3,3,NextProgGal,INTEGER,0,None,0


### Analysis

### Check Properties of massive galaxies

In [114]:
# select IDs of massive galaxies, with non 0 star formation from file 1

DirName_MR = '/net/bootes/scratch-ssd/SAM/test3/MR/'
database_file = DirName_MR + 'GalTree_file40.db'

query = '''select * from Henriques2015 
            where Snapnum = 58 and Type = 0 and StellarMass*1.e10/0.673>1e11 LIMIT 10''' 
df1 = run_query(database_file, query)

SSFR = np.log10(df1['Sfr']/(df1['StellarMass']*1e10/0.673))
sel = SSFR > -11.3
GalIds = df1.loc[sel,['HaloID','GalID','LastProgGal']]

print(GalIds)


#Select one tree from file 1
ii = 1

DirName_MR = '/net/bootes/scratch-ssd/SAM/test3/MR/'
database_file = DirName_MR + 'GalTree_file40.db'
initial_gal = GalIds['GalID'].iloc[ii]
query = '''select * from Henriques2015 
            where GalID>={:d} and GalID<={:d}'''.format(GalIds['GalID'].iloc[ii],GalIds['LastProgGal'].iloc[ii]) 
df_tree_1 = run_query(database_file, query)
print("read tree 1")

#And then Select same tree from file 2 
DirName_MR = '/net/bootes/scratch-ssd/SAM/test2/MR/'
database_file = DirName_MR + 'GalTree_file40.db'

query = '''select HaloID, GalId, LastProgGal
             from Henriques2015 
            where HaloID={:d} and Type=0'''.format(GalIds['HaloID'].iloc[ii]) 
GalIds_2 = run_query(database_file, query)

initial_gal_2 = GalIds_2['GalID'].iloc[0]
query = '''select * from Henriques2015 
            where GalID>={:d} and GalID<={:d}'''.format(GalIds_2['GalID'].iloc[0],GalIds_2['LastProgGal'].iloc[0]) 
df_tree_2 = run_query(database_file, query)

print("read tree 2")


#And then Select same tree from file 3 
DirName_MR = '/net/bootes/scratch-ssd/SAM/test1/MR/'
database_file = DirName_MR + 'GalTree_file40.db'

query = '''select HaloID, GalId, LastProgGal
             from Henriques2015 
            where HaloID={:d} and Type=0'''.format(GalIds['HaloID'].iloc[ii]) 
GalIds_3 = run_query(database_file, query)

initial_gal_3 = GalIds_3['GalID'].iloc[0]
query = '''select * from Henriques2015 
            where GalID>={:d} and GalID<={:d}'''.format(GalIds_3['GalID'].iloc[0],GalIds_3['LastProgGal'].iloc[0]) 
df_tree_3 = run_query(database_file, query)

print("read tree 3")

           HaloID           GalID     LastProgGal
2  40000001001023  40000001000005  40000001003524
5  40000004000005  40000004000005  40000004000774
7  40000006000005  40000006000005  40000006000709
9  40000008000005  40000008000005  40000008000890
read tree 1
read tree 2
read tree 3


In [115]:
xlim=[0,3]
#ylim=[-13,-7]
ylim=[9,12]
fig, ax = plt.subplots(figsize=(10,10))
ax.set_xlim(xlim), ax.set_ylim(ylim)

linestyle=['-',':','--']

for ii in range(0,3):
    if ii == 0:
        df_tree = df_tree_1
    if ii == 1:
        df_tree = df_tree_2
    if ii == 2:
        df_tree = df_tree_3
        
    Nsnaps = 64
    Redshift = np.zeros(Nsnaps,dtype=np.float32)    
    StellarMass = np.zeros(Nsnaps,dtype=np.float32)           
    SFR = np.zeros(Nsnaps,dtype=np.float32)   
    ColdGas = np.zeros(Nsnaps,dtype=np.float32)   
    H2fraction = np.zeros(Nsnaps,dtype=np.float32)   
    
    Nprogenitors=0  
    snap=0
    currentgal=initial_gal
    while True:       
        sel = (df_tree['GalID'] == currentgal)   
        Redshift[snap] = df_tree.loc[sel,'Redshift']
        StellarMass[snap] = df_tree.loc[sel,'StellarMass']*1.e10/Hubble_h
        SFR[snap] = df_tree.loc[sel,'Sfr']
        ColdGas[snap] = df_tree.loc[sel,'ColdGas']*1.e10/Hubble_h
        H2fraction[snap] = df_tree.loc[sel,'H2fraction']
           
        #stop if there are no more progenitors
        prog = df_tree.loc[df_tree['GalID']==currentgal,'FirstProgGal'].values[0]
  
        currentgal = prog
        if(prog==-1):
            break
        snap+=1  
    
    #ax.plot(Redshift,np.log10(SFR/StellarMass), linestyle=linestyle[ii], color='black')
    ax.plot(Redshift,np.log10(StellarMass), linestyle=linestyle[ii], color='red')
    ax.plot(Redshift,np.log10(ColdGas), linestyle=linestyle[ii], color='blue')
    ax.plot(Redshift,np.log10(ColdGas*H2fraction), linestyle=linestyle[ii], color='steelblue')




plt.savefig('./fig/galtree_plots.pdf')
plt.close()

print('plots done')

/export/data1/Workspace/GitHub_Python_plots/py3env/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in log10
/export/data1/Workspace/GitHub_Python_plots/py3env/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in log10
/export/data1/Workspace/GitHub_Python_plots/py3env/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: divide by zero encountered in log10


plots done


### Accretion History

In [7]:
file=Datadir+'Database_snapshots_table.fits'
fits_table=fits.open(file)
snap_table = fits_table[1]       
FullRedshiftList=snap_table.data['z'][::-1]
FullTimeList=snap_table.data['lookBackTime'][::-1]
    
FullTimeList=FullTimeList[FullRedshiftList>0.]
FullRedshiftList=FullRedshiftList[FullRedshiftList>0.]
   
plot_linestyle=[':','-.','--','-']

fig = plt.figure(figsize=(6,5))
grid = gridspec.GridSpec(2, 2)
grid.update(wspace=0.0, hspace=0.0)
        
subplot=plt.subplot()  
xlim=[-5.0,0.]
ylim=[-4.0, 0.0]
    
subplot.set_ylim(ylim), subplot.set_xlim(xlim)
    
xlab='$\log_{10}(M_{\mathrm{vir}}[\mathrm{M}_{\odot}])$'
ylab='Fraction'          
subplot.set_xlabel(xlab, fontsize=14), subplot.set_ylabel(ylab, fontsize=14)    
    
#11.0,12.0,13.0,14.0
HaloMassBins=np.arange(13.0,15.0,1.0)   

for kk in range (0,len(HaloMassBins)):   
  
    t1 = time.time()
    query = '''select * from Henriques2015 
                where Snapnum = 58 and Type = 0 and (Mvir*1.e10/{:0.2f})>{:0.2f} and (Mvir*1.e10/{:0.2f})<{:0.2f}
                LIMIT 10'''.format(Hubble_h, 10**(HaloMassBins[kk]-0.1),Hubble_h, 10**(HaloMassBins[kk]+0.1)) 
    df1 = run_query(database_file, query)
    print("time elapsed = %0.2fs" % (time.time()-t1))

    t1 = time.time()
    G0_MR=G_MR[(G_MR['SnapNum']==58) & (G_MR['Type']==0) & 
                (np.log10(G_MR['Mvir']*1.e10/Hubble_h)>HaloMassBins[kk]-0.1) & 
                (np.log10(G_MR['Mvir']*1.e10/Hubble_h)<HaloMassBins[kk]+0.1)]  
    print("time elapsed = %0.2fs" % (time.time()-t1))
    
    Ntrees=min(100,len(G0_MR))
        
    bin=0.2
    limits = [10.0,15.0]
    bin_arr = np.arange(limits[0],limits[1]+bin,bin)
    x_axis = bin_arr[0:len(bin_arr)-1]+bin/2.    
    Mvir_hist=np.zeros([len(bin_arr)-1,Ntrees],dtype=np.float32)    
    totalProgMass=np.zeros(Ntrees,dtype=np.float32)    
    FinalMass=np.zeros(Ntrees,dtype=np.float32)    

    print("")
    for ll in range(0,Ntrees):
        t1 = time.time()
        if(ll%(round(Ntrees/5.))==0):
            print("HaloMass[%0.1f], %d%% done" %(HaloMassBins[kk], ll/Ntrees*100))
        #ii = trees[ll]
        ii = ll
        
        initial_gal=G0_MR['GalID'][ii]   
        G0_MR_tree=G_MR[(G_MR['GalID']>=G0_MR['GalID'][ii]) & (G_MR['GalID']<=G0_MR['LastProgGal'][ii])]  
        FinalMass[ll]=np.log10(G0_MR['Mvir'][ii]*1.e10/Hubble_h)
            
        if(ll%(round(Ntrees/5.))==0):    
            print("time elapsed t1 = %0.5fs" % (time.time()-t1))    
            
            
        t2 = time.time()
        query = '''select * from Henriques2015 
                    where GalID>={:d} and GalID<={:d}'''.format(G0_MR['GalID'][ii],G0_MR['LastProgGal'][ii]) 
        df1 = run_query(database_file, query)
        if(ll%(round(Ntrees/5.))==0):    
            print("time elapsed t2= %0.2fs" % (time.time()-t2))    
            
        #skip tree with a single galaxy
        GalsInTree = len(G0_MR_tree)
        SnapsInTree = (1+G0_MR['SnapNum'][ii]-G_MR['SnapNum'][G_MR['GalID']==G0_MR['LastProgGal'][ii]])            
        if((GalsInTree)-1<=SnapsInTree):
            continue
           
        
        #count number of non main progenitors
        Nprogenitors=0  
        currentgal=initial_gal
        while True:    
            descendants = G0_MR_tree['DescendantGal']==currentgal
            if(len(G0_MR_tree['GalID'][descendants])>1):
                Nprogenitors+=(len(G0_MR_tree['GalID'][descendants])-1)                
            prog=G0_MR_tree['FirstProgGal'][G0_MR_tree['GalID']==currentgal]
            currentgal=prog               
            if prog==-1:
                break
        #print("Nprogenitors: %d" %(Nprogenitors))    
            
        Redshift=np.zeros(Nprogenitors,dtype=np.float32)    
        Mvir=np.zeros(Nprogenitors,dtype=np.float32)           
        MainGalMvir=np.zeros(len(FullRedshiftList),dtype=np.float32)        
            
        
        Nprogenitors=0  
        Nsnaps=0
        currentgal=initial_gal
        while True:    
            sel=G0_MR_tree['GalID']==currentgal
            MainGalMvir[Nsnaps]=np.log10(G0_MR_tree['Mvir'][sel]*1.e10/Hubble_h)
             
            #select all progenitor gals
            G_Prog_Gals=G0_MR_tree[(G0_MR_tree['DescendantGal']==currentgal)] 
            #select main progenitor
            FirstProgID = G0_MR_tree['FirstProgGal'][G0_MR_tree['GalID']==currentgal]  
            G_FirstProg = G0_MR_tree[G0_MR_tree['GalID']==FirstProgID]  
                    
            #CHANGE HERE
            #LOOP THROUGH ALL THE PROGENITORS TO FIND WHEN TEHY MERGED
                           
            #sel all non main progenitor gals
            FirstProgID = G0_MR_tree['FirstProgGal'][G0_MR_tree['GalID']==currentgal]   
            sel=G_Prog_Gals['GalID']!= FirstProgID                
            G_Prog_Gals=G_Prog_Gals[sel]
            #print(np.log10(Prog_Gals['Mvir']*1.e10/Hubble_h))
            progMass=0
            for jj in range(0,len(G_Prog_Gals)):
                Redshift[Nprogenitors+jj]=G_Prog_Gals['Redshift'][jj]  
                Mvir[Nprogenitors+jj]=np.log10(G_Prog_Gals['Mvir'][jj]*1.e10/Hubble_h)                       
                     
                progMass+=G_Prog_Gals['Mvir'][jj]*1.e10/Hubble_h
                    
            #Nprogenitors+=len(G0_MR_tree['GalID'][G0_MR_tree['DescendantGal']==currentgal])-1 
            Nprogenitors+=len(G_Prog_Gals)
               
            #stop if the main progenitor of current galaxy has Mvir<0.1 x final_Mvir
            if(np.log10(G_FirstProg['Mvir']*1.e10/Hubble_h) < (FinalMass[ll]-1.)):  
                break  
            
            #stop if there are no more progenitors
            prog=G0_MR_tree['FirstProgGal'][G0_MR_tree['GalID']==currentgal]
            currentgal=prog 
            if(prog==-1):
                break
            Nsnaps+=1  
                
                    
        #print(Mvir)
        hist_MR=np.histogram(Mvir, bins=bin_arr, range=(xlim[0],xlim[1]))          
        hist_MR=hist_MR[0]         
        #divide by bin value over (final_mass-0.1*final_mass) to get the percetange fraction of mass
        #in each bin
        Mass_dif = 10**FinalMass[ll]-10**(FinalMass[ll]-1.)
        Mvir_hist[:,ll]=hist_MR * (10**x_axis)/Mass_dif          
        totalProgMass[ll]=np.log10(np.sum(10**Mvir,axis=0))        
        
        if(ll%(round(Ntrees/5.))==0):
            print("total time elapsed = %0.5fs" % (time.time()-t1))
    #CHANGE HERE
    final_hist_mean=np.zeros(len(x_axis),dtype=np.float32)   
    final_hist_median=np.zeros(len(x_axis),dtype=np.float32)   
    for ii in range(0, len(x_axis)):
        final_hist_mean[ii] = np.mean(Mvir_hist[ii,:])
        final_hist_median[ii] = np.median(Mvir_hist[ii,:])
       
    subplot.plot(x_axis-HaloMassBins[kk], np.log10(final_hist_mean), linestyle=plot_linestyle[kk], linewidth=2, color='gray')
    #subplot.plot(x_axis, np.log10(final_hist_median), linestyle=plot_linestyle[kk], linewidth=2, color='gray')
    print("Mass in mergers=%0.2f" % sum(final_hist_mean))
       
plt.tight_layout()   
#current_function =  inspect.getframeinfo(inspect.currentframe()).function   
#plt.savefig('./fig/plots_'+current_function+'.pdf')
plt.savefig('./fig/plots_accretion_history.pdf')
plt.close()        
   
print("done")    

time elapsed = 0.05s
time elapsed = 0.28s

HaloMass[13.0], 0% done
time elapsed t1 = 0.04513s
time elapsed t2= 0.01s
total time elapsed = 0.08380s
HaloMass[13.0], 19% done
time elapsed t1 = 0.04301s
time elapsed t2= 0.01s
total time elapsed = 0.07787s
HaloMass[13.0], 38% done
time elapsed t1 = 0.03736s
time elapsed t2= 0.01s
total time elapsed = 0.06457s
HaloMass[13.0], 57% done
time elapsed t1 = 0.03749s
time elapsed t2= 0.01s
total time elapsed = 0.06752s
HaloMass[13.0], 77% done
time elapsed t1 = 0.03689s
time elapsed t2= 0.01s
total time elapsed = 0.06604s
HaloMass[13.0], 96% done
time elapsed t1 = 0.03764s
time elapsed t2= 0.01s
total time elapsed = 0.07093s
Mass in mergers=0.27


/export/data1/Workspace/GitHub_Python_plots/py3env/lib/python3.6/site-packages/ipykernel_launcher.py:163: RuntimeWarning: divide by zero encountered in log10


time elapsed = 0.31s
time elapsed = 0.13s

HaloMass[14.0], 0% done
time elapsed t1 = 0.03628s
time elapsed t2= 0.03s
total time elapsed = 0.09013s
HaloMass[14.0], 25% done
time elapsed t1 = 0.04368s
time elapsed t2= 0.02s
total time elapsed = 0.08097s
HaloMass[14.0], 50% done
time elapsed t1 = 0.03606s
time elapsed t2= 0.02s
total time elapsed = 0.08159s
HaloMass[14.0], 75% done
time elapsed t1 = 0.03595s
time elapsed t2= 0.02s
total time elapsed = 0.07706s
Mass in mergers=0.25
done


In [ ]:
def accretion_history(G_MR, Volume_MR):
   
    file=Datadir+'Database_snapshots_table.fits'
    fits_table=fits.open(file)
    snap_table = fits_table[1]       
    FullRedshiftList=snap_table.data['z'][::-1]
    FullTimeList=snap_table.data['lookBackTime'][::-1]
    
    FullTimeList=FullTimeList[FullRedshiftList>0.]
    FullRedshiftList=FullRedshiftList[FullRedshiftList>0.]
   
    #FullRedshiftList=FullTimeList
    
    plot_linestyle=[':','-.','--','-']

    fig = plt.figure(figsize=(two_one_size_small[0]+1.0,two_one_size_small[1]))
    grid = gridspec.GridSpec(2, 2)
    grid.update(wspace=0.0, hspace=0.0)
    
    '''
    
    #HaloMassBins=np.arange(11.0,15.0,1.0)
    HaloMassBins=np.arange(14.0,15.0,1.0)
    for kk in range (0,len(HaloMassBins)):
    
        G0_MR=G_MR[(G_MR['SnapNum']==58) & (G_MR['Type']==0) & 
                   (np.log10(G_MR['Mvir']*1.e10/Hubble_h)>HaloMassBins[kk]-0.1) & 
                   (np.log10(G_MR['Mvir']*1.e10/Hubble_h)<HaloMassBins[kk]+0.1)]  
                 
        Ntrees=min(1,len(G0_MR))
        
        print('Mass=%0.1f Ntrees=%d selected=%d' % (HaloMassBins[kk], len(G0_MR), Ntrees))
    
        bin=0.5
        bin_arr=np.arange(xlim[0],xlim[1]+bin,bin)
        x_axis=bin_arr[0:len(bin_arr)-1]+bin/2.    
        Mvir_hist=np.zeros([len(bin_arr)-1,Ntrees],dtype=np.float32)    
        totalProgMass=np.zeros(Ntrees,dtype=np.float32)    
        FinalMass=np.zeros(Ntrees,dtype=np.float32)    
    
        for ii in range(0,Ntrees):
            initial_gal=G0_MR['GalID'][ii]   
            G0_MR_tree=G_MR[(G_MR['GalID']>=G0_MR['GalID'][ii]) & (G_MR['GalID']<=G0_MR['LastProgGal'][ii])]  
            FinalMass[ii]=np.log10(G0_MR['Mvir'][ii]*1.e10/Hubble_h)
            
            #count number of non main progenitors
            Nprogenitors=0  
            currentgal=initial_gal
            while True:    
                if(len(G0_MR_tree['GalID'][G0_MR_tree['DescendantGal']==currentgal])>1):
                    Nprogenitors+=len(G0_MR_tree['GalID'][G0_MR_tree['DescendantGal']==currentgal])-1                 
                prog=G0_MR_tree['FirstProgGal'][G0_MR_tree['GalID']==currentgal]
                currentgal=prog               
                if prog==-1:
                    break
                
            
            Redshift=np.zeros(Nprogenitors,dtype=np.float32)    
            Mvir=np.zeros(Nprogenitors,dtype=np.float32)
            ColdGas=np.zeros(Nprogenitors,dtype=np.float32)
            StellarMass=np.zeros(Nprogenitors,dtype=np.float32)
            BlackHoleMass=np.zeros(Nprogenitors,dtype=np.float32)
            SFR=np.zeros(Nprogenitors,dtype=np.float32)

            MainGalMvir=np.zeros(len(FullRedshiftList),dtype=np.float32)        
            MainGalColdGas=np.zeros(len(FullRedshiftList),dtype=np.float32)
            MainGalStellarMass=np.zeros(len(FullRedshiftList),dtype=np.float32)
            MainGalBlackHoleMass=np.zeros(len(FullRedshiftList),dtype=np.float32)
            MainGalSFR=np.zeros(len(FullRedshiftList),dtype=np.float32)
        
            Nprogenitors=0  
            Nsnaps=0
            currentgal=initial_gal
            while True:    
                sel=G0_MR_tree['GalID']==currentgal
                MainGalMvir[Nsnaps]=np.log10(G0_MR_tree['Mvir'][sel]*1.e10/Hubble_h)
                MainGalColdGas[Nsnaps]=np.log10(G0_MR_tree['ColdGas'][sel]*1.e10/Hubble_h)
                MainGalStellarMass[Nsnaps]=np.log10(G0_MR_tree['StellarMass'][sel]*1.e10/Hubble_h)            
                MainGalBlackHoleMass[Nsnaps]=np.log10(G0_MR_tree['BlackHoleMass'][sel]*1.e10/Hubble_h)
                #print(G0_MR_tree['Redshift'][sel])
                #print("MainGalMass=%0.5e" % 10**MainGalMvir[Nsnaps])
                if(len(G0_MR_tree['GalID'][G0_MR_tree['DescendantGal']==currentgal])>1):
                    #select all progenitor gals
                    Prog_Gals=G0_MR_tree[(G0_MR_tree['DescendantGal']==currentgal)] 
                    #print(np.log10(Prog_Gals['Mvir']*1.e10/Hubble_h))
                    #sel main progenitor
                    sel=Prog_Gals['GalID']==G0_MR_tree['FirstProgGal'][G0_MR_tree['GalID']==currentgal]
                
                    #sel all non main progenitor gals
                    sel=Prog_Gals['GalID']!=G0_MR_tree['FirstProgGal'][G0_MR_tree['GalID']==currentgal]                    
                    Prog_Gals=Prog_Gals[sel]
                    #print(np.log10(Prog_Gals['Mvir']*1.e10/Hubble_h))
                    progMass=0
                    for jj in range(0,len(Prog_Gals)):
                        Redshift[Nprogenitors+jj]=Prog_Gals['Redshift'][jj]        
                        Mvir[Nprogenitors+jj]=np.log10(Prog_Gals['Mvir'][jj]*1.e10/Hubble_h)
                        ColdGas[Nprogenitors+jj]=np.log10(Prog_Gals['ColdGas'][jj]*1.e10/Hubble_h)
                        StellarMass[Nprogenitors+jj]=np.log10(Prog_Gals['StellarMass'][jj]*1.e10/Hubble_h)
                        BlackHoleMass[Nprogenitors+jj]=np.log10(Prog_Gals['BlackHoleMass'][jj]*1.e10/Hubble_h)
                        SFR[Nprogenitors+jj]=np.log10(Prog_Gals['Sfr'][jj])
                        progMass+=Prog_Gals['Mvir'][jj]*1.e10/Hubble_h
                    
                    #Nprogenitors+=len(G0_MR_tree['GalID'][G0_MR_tree['DescendantGal']==currentgal])-1 
                    Nprogenitors+=len(Prog_Gals)
                #print("progMass=%0.5e" % progMass)
                #print("ratio=%0.5f\n" % (progMass/10**MainGalMvir[Nsnaps]))
                
                prog=G0_MR_tree['FirstProgGal'][G0_MR_tree['GalID']==currentgal]
                currentgal=prog    
                Nsnaps+=1  
                if prog==-1:
                    break
                    
        
            hist_MR=np.histogram(Mvir, bins=bin_arr, range=(xlim[0],xlim[1]))          
            hist_MR=hist_MR[0]         
            #Mvir_hist[:,ii]=hist_MR/len(Mvir)
            Mvir_hist[:,ii]=hist_MR/len(Mvir)#*(10**x_axis/1.e14)
            totalProgMass[ii]=np.sum(10**Mvir,axis=0)        
        
            #print("tot prog mass=%0.1f" % totalProgMass[ii])
        
        (x_array,median,mean,pc16,pc84,rms)=convert_2d_array_into_region(x_axis,Mvir_hist)
        subplot.fill_between(x_array,np.log10(pc84),np.log10(pc16),interpolate=True,alpha=0.4,
                             facecolor='gray',edgecolor='gray')         
        subplot.plot(x_array,np.log10(median), linestyle=plot_linestyle[kk], linewidth=2, color='gray')
    
        #print(np.median(totalProgMass/10**(FinalMass)))
        
        #subplot.plot(x_axis,Mvir_hist[:,0], color='red', linewidth=2, linestyle=':', drawstyle='steps') '''
         
    subplot=plt.subplot()  
    #xlim=[8.5,15.]
    xlim=[10.5,15.]
    xlim=[-5.0,0.]
    ylim=[-4.0, 0.0]
    
    subplot.set_ylim(ylim), subplot.set_xlim(xlim)
    
    #format axis
    #majorFormatter = FormatStrFormatter('%d')
    #subplot.xaxis.set_major_locator(MultipleLocator(2.0))    
    #subplot.xaxis.set_minor_locator(MultipleLocator(0.5)) 
    #subplot.yaxis.set_major_locator(MultipleLocator(2.0))    
    #subplot.yaxis.set_minor_locator(MultipleLocator(0.5)) 
   
    xlab='$\log_{10}(M_{\mathrm{vir}}[\mathrm{M}_{\odot}])$'
    ylab='Fraction'          
    subplot.set_xlabel(xlab, fontsize=14), subplot.set_ylabel(ylab, fontsize=14)    
    
    #11.0,12.0,13.0,14.0
    HaloMassBins=np.arange(11.0,15.0,1.0)
    HaloMassBins=np.arange(12.0,15.0,1.0)    
    for kk in range (0,len(HaloMassBins)):   
        G0_MR=G_MR[(G_MR['SnapNum']==58) & (G_MR['Type']==0) & 
                   (np.log10(G_MR['Mvir']*1.e10/Hubble_h)>HaloMassBins[kk]-0.1) & 
                   (np.log10(G_MR['Mvir']*1.e10/Hubble_h)<HaloMassBins[kk]+0.1)]  
                 
        Ntrees=min(100,len(G0_MR))
        
       
        '''trees=[]
        #find trees that have more than a single halo
        for ii in range(0,Ntrees):
            initial_gal=G0_MR['GalID'][ii]   
            G0_MR_tree=G_MR[(G_MR['GalID']>=G0_MR['GalID'][ii]) & (G_MR['GalID']<=G0_MR['LastProgGal'][ii])]  
                      
            GalsInTree = len(G0_MR_tree)
            SnapsInTree = (1+G0_MR['SnapNum'][ii]-G_MR['SnapNum'][G_MR['GalID']==G0_MR['LastProgGal'][ii]])
            #print((GalsInTree-1),SnapsInTree)
            if((GalsInTree-1)<=SnapsInTree):
                continue
            else:                
                trees.append(ii)
                
        Ntrees = len(trees)       
        print('Mass=%0.1f Ntrees=%d selected=%d' % (HaloMassBins[kk], len(G0_MR), Ntrees))'''
        
        bin=0.2
        limits = [10.0,15.0]
        bin_arr = np.arange(limits[0],limits[1]+bin,bin)
        x_axis = bin_arr[0:len(bin_arr)-1]+bin/2.    
        Mvir_hist=np.zeros([len(bin_arr)-1,Ntrees],dtype=np.float32)    
        totalProgMass=np.zeros(Ntrees,dtype=np.float32)    
        FinalMass=np.zeros(Ntrees,dtype=np.float32)    
    
        for ll in range(0,Ntrees):
            #ii = trees[ll]
            ii = ll
            initial_gal=G0_MR['GalID'][ii]   
            G0_MR_tree=G_MR[(G_MR['GalID']>=G0_MR['GalID'][ii]) & (G_MR['GalID']<=G0_MR['LastProgGal'][ii])]  
            FinalMass[ll]=np.log10(G0_MR['Mvir'][ii]*1.e10/Hubble_h)
            
            #skip tree with a single galaxy
            GalsInTree = len(G0_MR_tree)
            SnapsInTree = (1+G0_MR['SnapNum'][ii]-G_MR['SnapNum'][G_MR['GalID']==G0_MR['LastProgGal'][ii]])            
            if((GalsInTree)-1<=SnapsInTree):
                continue
           
        
            #count number of non main progenitors
            Nprogenitors=0  
            currentgal=initial_gal
            while True:    
                descendants = G0_MR_tree['DescendantGal']==currentgal
                if(len(G0_MR_tree['GalID'][descendants])>1):
                    Nprogenitors+=(len(G0_MR_tree['GalID'][descendants])-1)                
                prog=G0_MR_tree['FirstProgGal'][G0_MR_tree['GalID']==currentgal]
                currentgal=prog               
                if prog==-1:
                    break
            #print("Nprogenitors: %d" %(Nprogenitors))    
            
            Redshift=np.zeros(Nprogenitors,dtype=np.float32)    
            Mvir=np.zeros(Nprogenitors,dtype=np.float32)           
            MainGalMvir=np.zeros(len(FullRedshiftList),dtype=np.float32)        
            
        
            Nprogenitors=0  
            Nsnaps=0
            currentgal=initial_gal
            while True:    
                sel=G0_MR_tree['GalID']==currentgal
                MainGalMvir[Nsnaps]=np.log10(G0_MR_tree['Mvir'][sel]*1.e10/Hubble_h)
             
                #select all progenitor gals
                G_Prog_Gals=G0_MR_tree[(G0_MR_tree['DescendantGal']==currentgal)] 
                #select main progenitor
                FirstProgID = G0_MR_tree['FirstProgGal'][G0_MR_tree['GalID']==currentgal]  
                G_FirstProg = G0_MR_tree[G0_MR_tree['GalID']==FirstProgID]  
                    
                #CHANGE HERE
                #LOOP THROUGH ALL THE PROGENITORS TO FIND WHEN TEHY MERGED
                           
                #sel all non main progenitor gals
                FirstProgID = G0_MR_tree['FirstProgGal'][G0_MR_tree['GalID']==currentgal]   
                sel=G_Prog_Gals['GalID']!= FirstProgID                
                G_Prog_Gals=G_Prog_Gals[sel]
                #print(np.log10(Prog_Gals['Mvir']*1.e10/Hubble_h))
                progMass=0
                for jj in range(0,len(G_Prog_Gals)):
                    Redshift[Nprogenitors+jj]=G_Prog_Gals['Redshift'][jj]  
                    Mvir[Nprogenitors+jj]=np.log10(G_Prog_Gals['Mvir'][jj]*1.e10/Hubble_h)                       
                     
                    progMass+=G_Prog_Gals['Mvir'][jj]*1.e10/Hubble_h
                    
                #Nprogenitors+=len(G0_MR_tree['GalID'][G0_MR_tree['DescendantGal']==currentgal])-1 
                Nprogenitors+=len(G_Prog_Gals)
               
                #stop if the main progenitor of current galaxy has Mvir<0.1 x final_Mvir
                if(np.log10(G_FirstProg['Mvir']*1.e10/Hubble_h) < (FinalMass[ll]-1.)):  
                    break  
            
                #stop if there are no more progenitors
                prog=G0_MR_tree['FirstProgGal'][G0_MR_tree['GalID']==currentgal]
                currentgal=prog 
                if(prog==-1):
                    break
                Nsnaps+=1  
                
                
                
                  
                
                #if prog==-1:
                #    break
                    
            #print(Mvir)
            hist_MR=np.histogram(Mvir, bins=bin_arr, range=(xlim[0],xlim[1]))          
            hist_MR=hist_MR[0]         
            #divide by bin value over (final_mass-0.1*final_mass) to get the percetange fraction of mass
            #in each bin
            Mass_dif = 10**FinalMass[ll]-10**(FinalMass[ll]-1.)
            Mvir_hist[:,ll]=hist_MR * (10**x_axis)/Mass_dif          
            totalProgMass[ll]=np.log10(np.sum(10**Mvir,axis=0))        
        
        #for ll in range(0,Ntrees):
            #print(Mvir_hist[:,ii])
            #print(10**totalProgMass[ll]/10**FinalMass[ll])
        
        #CHANGE HERE
        final_hist_mean=np.zeros(len(x_axis),dtype=np.float32)   
        final_hist_median=np.zeros(len(x_axis),dtype=np.float32)   
        for ii in range(0, len(x_axis)):
            final_hist_mean[ii] = np.mean(Mvir_hist[ii,:])
            final_hist_median[ii] = np.median(Mvir_hist[ii,:])
       
        subplot.plot(x_axis-HaloMassBins[kk], np.log10(final_hist_mean), linestyle=plot_linestyle[kk], linewidth=2, color='gray')
        #subplot.plot(x_axis, np.log10(final_hist_median), linestyle=plot_linestyle[kk], linewidth=2, color='gray')
        print("Mass in mergers=%0.2f" % sum(final_hist_mean))
        
       
        #subplot.plot(np.log10((10**x_axis/10**HaloMassBins[kk])), np.log10(final_hist_mean), 
        #             linestyle=plot_linestyle[kk], linewidth=2, color='gray')
         
    plt.tight_layout()   
    current_function =  inspect.getframeinfo(inspect.currentframe()).function   
    plt.savefig('./fig/plots_'+current_function+'.pdf')
    plt.close()        
    
    plot_prog_at_single_redshift=0
    
    if(plot_prog_at_single_redshift==1):
            
        subplot=plt.subplot()   
        xlim=[-5.0,0.]   
        ylim=[-3., 0.0]
        subplot.set_ylim(ylim), subplot.set_xlim(xlim)
    
        xlab='$\log_{10}(M_{\mathrm{vir}}[\mathrm{M}_{\odot}])$'
        ylab='Fraction'          
        subplot.set_xlabel(xlab, fontsize=14), subplot.set_ylabel(ylab, fontsize=14)    
    
        #11.0,12.0,13.0,14.0   
        HaloMassBins=np.arange(12.0,15.0,1.0)    
        
        for kk in range (0,len(HaloMassBins)):
    
            G0_MR=G_MR[(G_MR['SnapNum']==58) & (G_MR['Type']==0) & 
                       (np.log10(G_MR['Mvir']*1.e10/Hubble_h)>HaloMassBins[kk]-0.1) & 
                       (np.log10(G_MR['Mvir']*1.e10/Hubble_h)<HaloMassBins[kk]+0.1)]  
                 
            Ntrees=min(10,len(G0_MR))
            print(Ntrees)
       
          
            totalProgMass=np.zeros(Ntrees,dtype=np.float32)  
            target_redshift=np.zeros(Ntrees,dtype=np.float32)    
            target_mass=np.zeros(Ntrees,dtype=np.float32)    
            FinalMass=np.zeros(Ntrees,dtype=np.float32)    
    
            for ll in range(0,Ntrees):           
                initial_galID = G0_MR['GalID'][ll]   
                FinalMass[ll] = np.log10(G0_MR['Mvir'][ll]*1.e10/Hubble_h)
                G_tree = G_MR[(G_MR['GalID']>=G0_MR['GalID'][ll]) & (G_MR['GalID']<=G0_MR['LastProgGal'][ll])]  
              
                currentgalID = initial_galID
                while True:    
                
                    FirstProgID = G_tree['FirstProgGal'][G_tree['GalID']==currentgalID]
                    G_FirstProg = G_tree[G_tree['GalID']==FirstProgID]
                
                    #if main progenitor Mvir = 0.1 x Mvir(z=0)   
                    if(FirstProgID>-1):                    
                        if(np.log10(G_FirstProg['Mvir']*1.e10/Hubble_h) < (FinalMass[ll]-1.)):                       
                            target_redshift[ll] = G_FirstProg['Redshift']
                            target_mass[ll] = np.log10(G_FirstProg['Mvir']*1.e10/Hubble_h)
                            break  
                    #if there are no more progenitors, break        
                    else:
                        break
               
                    currentgalID=FirstProgID    
                    Nsnaps+=1  
                
            bin=0.2
            hist_lim = [10.0,15.0]   
            bin_arr=np.arange(hist_lim[0],hist_lim[1]+bin,bin)
            Mvir_hist=np.zeros([len(bin_arr)-1,Ntrees],dtype=np.float32)       
        
            for ll in range(0,Ntrees):
                initial_galID = G0_MR['GalID'][ll]   
           
                G_target_snap = G_MR[(G_MR['GalID']>=G0_MR['GalID'][ll]) & 
                                     (G_MR['GalID']<=G0_MR['LastProgGal'][ll]) &
                                     (G_MR['Type']<2) & (G_MR['Redshift']==target_redshift[ll])]  
            
            
                hist_MR=np.histogram(np.log10(G_target_snap['Mvir']*1.e10/Hubble_h), bins=bin_arr)      
                x_axis = hist_MR[1][0:len(hist_MR[1][:])-1]+bin/2.               
                Mvir_hist[:,ll] = hist_MR[0]*((10**x_axis)/(10**FinalMass[ll]-10**target_mass[ll]))
           
        
            final_hist_mean=np.zeros(len(bin_arr)-1,dtype=np.float32)   
            final_hist_median=np.zeros(len(bin_arr)-1,dtype=np.float32)   
            for ii in range(0, len(x_axis)):
                final_hist_mean[ii] = np.mean(Mvir_hist[ii,:])
                final_hist_median[ii] = np.median(Mvir_hist[ii,:])
          
        
            subplot.plot(np.log10((10**x_axis/10**HaloMassBins[kk])),np.log10(final_hist_mean), linestyle=plot_linestyle[kk], linewidth=2, color='gray')
       
     
        plt.tight_layout()   
        current_function =  inspect.getframeinfo(inspect.currentframe()).function   
        plt.savefig('./fig/plots_'+current_function+'_2.pdf')
        plt.close()
    
    
    #QUERIES FOR HALOS
    
    '''select top 10 log10(M_crit200*1.e10/0.67), 
                  HaloID        
    from MPAHaloTrees..MrscPlanck1        
    where M_crit200>10000. and snapnum=58'''
    
    '''HaloID    LAstProg
    84000007005505	84000007008523'''
    
    return 
#accretion_history